# Проект 16 создание модели прогнозирования оттока клиентов из телеком компании

### Условия задачи:

Оператор связи «Ниединогоразрыва.ком» хочет научиться прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.

### Описание услуг

Оператор предоставляет два основных типа услуг: 

1. Стационарную телефонную связь. Возможно подключение телефонного аппарата к нескольким линиям одновременно.
2. Интернет. Подключение может быть двух типов: через телефонную линию (DSL*,* от англ. *digital subscriber line*, «цифровая абонентская линия») или оптоволоконный кабель (*Fiber optic*).  

Также доступны такие услуги:

- Интернет-безопасность: антивирус (*DeviceProtection*) и блокировка небезопасных сайтов (*OnlineSecurity*);
- Выделенная линия технической поддержки (*TechSupport*);
- Облачное хранилище файлов для резервного копирования данных (*OnlineBackup*);
- Стриминговое телевидение (*StreamingTV*) и каталог фильмов (*StreamingMovies*).

За услуги клиенты могут платить каждый месяц или заключить договор на 1–2 года. Доступны различные способы расчёта и возможность получения электронного чека.

### Описание данных

Данные состоят из файлов, полученных из разных источников:

- `contract.csv` — информация о договоре;
- `personal.csv` — персональные данные клиента;
- `internet.csv` — информация об интернет-услугах;
- `phone.csv` — информация об услугах телефонии.

Во всех файлах столбец `customerID` содержит код клиента.

Информация о договорах актуальна на 1 февраля 2020.

### Цели:

Целевой признак: столбец `'EndDate'` равен `'No'`.

Основная метрика: AUC-ROC.

Дополнительная метрика: Accuracy.

Критерии оценки:

- AUC-ROC < 0.75 — 0 sp
- 0.75 ≤ AUC-ROC < 0.81 — 4 sp
- 0.81 ≤ AUC-ROC < 0.85 — 4.5 sp
- 0.85 ≤ AUC-ROC < 0.87 — 5 sp
- 0.87 ≤ AUC-ROC < 0.88 — 5.5 sp
- AUC-ROC ≥ 0.88 — 6 sp

# Начало работы над проектом:

In [23]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [24]:
contract = pd.read_csv('/datasets/final_provider/contract.csv')
personal = pd.read_csv('/datasets/final_provider/personal.csv')
internet = pd.read_csv('/datasets/final_provider/internet.csv')
phone = pd.read_csv('/datasets/final_provider/phone.csv')

### Анализ contract:

In [25]:
contract 
# id, дата начала договора, дата конца договора, период оплаты, безбумажное выставление счетов, способ оплаты,
# eжемесячные платежи, общие расходы

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65
...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6


In [26]:
contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
customerID          7043 non-null object
BeginDate           7043 non-null object
EndDate             7043 non-null object
Type                7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


### Анализ personal:

In [27]:
personal
# id, пол, пожилой, партнёр, Иждивенцы

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No
...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes
7039,2234-XADUH,Female,0,Yes,Yes
7040,4801-JZAZL,Female,0,Yes,Yes
7041,8361-LTMKD,Male,1,Yes,No


In [28]:
personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
customerID       7043 non-null object
gender           7043 non-null object
SeniorCitizen    7043 non-null int64
Partner          7043 non-null object
Dependents       7043 non-null object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


### Анализ internet:

##### Описание услуг:

Оператор предоставляет два основных типа услуг: 

1. Стационарную телефонную связь. Возможно подключение телефонного аппарата к нескольким линиям одновременно.
2. Интернет. Подключение может быть двух типов: через телефонную линию (DSL*,* от англ. *digital subscriber line*, «цифровая абонентская линия») или оптоволоконный кабель (*Fiber optic*).

##### Также доступны такие услуги:

- Интернет-безопасность: антивирус (*DeviceProtection*) и блокировка небезопасных сайтов (*OnlineSecurity*);
- Выделенная линия технической поддержки (*TechSupport*);
- Облачное хранилище файлов для резервного копирования данных (*OnlineBackup*);
- Стриминговое телевидение (*StreamingTV*) и каталог фильмов (*StreamingMovies*).

In [29]:
internet
# id

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...
5512,6840-RESVB,DSL,Yes,No,Yes,Yes,Yes,Yes
5513,2234-XADUH,Fiber optic,No,Yes,Yes,No,Yes,Yes
5514,4801-JZAZL,DSL,Yes,No,No,No,No,No
5515,8361-LTMKD,Fiber optic,No,No,No,No,No,No


In [30]:
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
customerID          5517 non-null object
InternetService     5517 non-null object
OnlineSecurity      5517 non-null object
OnlineBackup        5517 non-null object
DeviceProtection    5517 non-null object
TechSupport         5517 non-null object
StreamingTV         5517 non-null object
StreamingMovies     5517 non-null object
dtypes: object(8)
memory usage: 344.9+ KB


### Анализ phone:

In [31]:
phone
# id, несколько линий

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes
...,...,...
6356,2569-WGERO,No
6357,6840-RESVB,Yes
6358,2234-XADUH,Yes
6359,8361-LTMKD,Yes


In [32]:
phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
customerID       6361 non-null object
MultipleLines    6361 non-null object
dtypes: object(2)
memory usage: 99.5+ KB


# Примерный план работы:

1. Проанализировать датасеты, которые нам предоставили
2. Понять, как склеивать таблицы и что делать с пропусками, которые образуются. Понять - как лучше заполнить пропуски.
3. Понять, как предобработать данные для улучшения целевой метрики у модели (маштабирование, определение категориальных признаков, сделать балансировку классов)
4. Аналитически выбрать наиболее подходящую/ие модель/модели для данной задачи.
5. Обучить модель/модели, перебирать гиперапараметры, чтобы добиться лучшего значения метрики.

# Создание датасета для модели:

In [60]:
# начнём создавать датасет для создания модели:
full_data = contract.copy()

#заменим index на customerID, для будущего удобства слияния таблиц, так как слияние происходит по индексам
full_data.index = full_data['customerID']

#выделим время:
full_data['BeginDate'] = pd.to_datetime(full_data['BeginDate'], format='%Y-%m-%d')

full_data.loc[full_data.index,'date_today'] = ['2020-02-01']
full_data['date_today'] = pd.to_datetime(full_data['date_today'], format='%Y-%m-%d')

#full_data['BeginDate_year']= pd.DatetimeIndex(full_data['BeginDate']).year

#full_data['BeginDate_month']= pd.DatetimeIndex(full_data['BeginDate']).month

#full_data['BeginDate_day']= pd.DatetimeIndex(full_data['BeginDate']).day #везде первый день, бесполезный признак

# Узнаем, сколько месяцев пользовались нашими услугами до расторжения договора:
have_enddate = full_data.loc[full_data['EndDate'] != 'No'].copy()
have_enddate['EndDate'] = pd.to_datetime(have_enddate['EndDate'], format='%Y-%m-%d %H:%M:%S')

have_enddate['EndDate_year'] = pd.DatetimeIndex(have_enddate['EndDate']).year

have_enddate['EndDate_month'] = pd.DatetimeIndex(have_enddate['EndDate']).month

#разница:
have_enddate['duration_contract'] = have_enddate['EndDate'] - full_data['BeginDate']

# если существует дата окончания договора, заменим значение на 1, это подразумевает, что дата окончания договора сущесвует:
index_have_enddate = full_data.loc[full_data['EndDate'] != 'No'].index
full_data.loc[index_have_enddate,'EndDate'] = 1

# если не существует дата окончания договора, заменим значение на 0, это подразумевает, что дата окончания договора не сущесвует:
index_no_enddate = full_data.loc[full_data['EndDate'] == 'No'].index
full_data.loc[index_no_enddate,'EndDate'] = 0

full_data.loc[index_have_enddate,'duration_contract'] = have_enddate['duration_contract']
full_data.loc[index_no_enddate,'duration_contract'] = full_data['date_today'] - full_data['BeginDate']

# удалим столбец, которым мы заменили индекс (пока рано решать какие ещё столбцы бесполезные и их стоит удалить тоже): 
full_data = full_data.drop(['customerID','BeginDate','TotalCharges','date_today'], axis='columns')

#full_data['duration_contract'] = have_enddate['duration_contract']
#full_data['EndDate_year'] = have_enddate['EndDate_year']
#full_data['EndDate_month'] = have_enddate['EndDate_month']

#full_data

In [61]:
# Предподготовка personal для слияния:
add_data0 = personal.copy()

#заменим index на customerID, для будущего удобства слияния таблиц, так как слияние происходит по индексам
add_data0.index = add_data0['customerID']

# удалим столбец, которым мы заменили индекс (пока рано решать какие ещё столбцы бесполезные и их стоит удалить тоже): 
add_data0 = add_data0.drop(['customerID'], axis='columns')

#add_data0

In [62]:
# Слияние contract и personal:

full_data['gender'] = add_data0['gender']
full_data['SeniorCitizen'] = add_data0['SeniorCitizen']
full_data['Partner'] = add_data0['Partner']
full_data['Dependents'] = add_data0['Dependents']

#full_data

In [63]:
# Предподготовка internet для слияния:
add_data1 = internet.copy()

#заменим index на customerID, для будущего удобства слияния таблиц, так как слияние происходит по индексам
add_data1.index = add_data1['customerID']

# удалим столбец, которым мы заменили индекс (пока рано решать какие ещё столбцы бесполезные и их стоит удалить тоже): 
add_data1 = add_data1.drop(['customerID'], axis='columns')

#add_data1

In [64]:
# Слияние contract, personal c  internet:

full_data['InternetService'] = add_data1['InternetService']

full_data = full_data.fillna('unknown')

full_data['OnlineSecurity'] = add_data1['OnlineSecurity']
full_data['OnlineBackup'] = add_data1['OnlineBackup']
full_data['DeviceProtection'] = add_data1['DeviceProtection']
full_data['TechSupport'] = add_data1['TechSupport']
full_data['StreamingTV'] = add_data1['StreamingTV']
full_data['StreamingMovies'] = add_data1['StreamingMovies']

full_data = full_data.fillna('No') #если клиентов нет в базе, вероятно они не пользуются услугами

#full_data

In [65]:
# Предподготовка phone для слияния:
add_data2 = phone.copy()

#заменим index на customerID, для будущего удобства слияния таблиц, так как слияние происходит по индексам
add_data2.index = add_data2['customerID']

# удалим столбец, которым мы заменили индекс (пока рано решать какие ещё столбцы бесполезные и их стоит удалить тоже): 
add_data2 = add_data2.drop(['customerID'], axis='columns')

#add_data2

In [66]:
# Слияние contract, personal,  internet c phone:

full_data['MultipleLines'] = add_data2['MultipleLines'] 

full_data = full_data.fillna('unknown')

# Смотрим новосозданную таблицу слияния, в ней будет много NaN, так как:
# internet и phone имели меньшую длину датафрейма, чем contract и personal

# Создание модели:

In [67]:
cat_features = ['Type','PaperlessBilling','PaymentMethod',
                'gender','SeniorCitizen','Partner','Dependents',
                'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection',
                'TechSupport','StreamingTV','StreamingMovies','MultipleLines']


from sklearn import preprocessing
#Перевод текстовых свойств в числовой формат:

columns_coding = ['Type','PaperlessBilling','PaymentMethod',
                'gender','Partner','Dependents',
                'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection',
                  'TechSupport','StreamingTV','StreamingMovies','MultipleLines']

label_encoder = preprocessing.LabelEncoder()
for colum in columns_coding:
    label_encoder.fit(full_data[colum])
    full_data[colum] = label_encoder.transform(full_data[colum])
#full_data

In [68]:
# отмаштабируем признаки:

numeric = ['MonthlyCharges', 'duration_contract']

full_data['duration_contract'] = pd.to_numeric(full_data['duration_contract'], errors='coerce')

for column in numeric:
    max_column = full_data[column].max()
    full_data[column] = full_data[column]/max_column

#
#full_data

In [69]:
# борьба с дисбалансом классов для бинарной классификации:
from sklearn.utils import shuffle
def upsample(data, target):
    imbalance = data[target].value_counts()
    coefficient = int(round(imbalance[0]/imbalance[1]))
    
    index_data_zeros = data.loc[data[target] == 0].index
    data_zeros = data.loc[index_data_zeros]
    
    index_data_ones = data.loc[data[target] == 1].index
    data_ones = data.loc[index_data_ones]
    
    data_upsampled = pd.concat([data_zeros] + [data_ones] * coefficient)
    final_data_upsampled = shuffle(data_upsampled, random_state=12345)
    return final_data_upsampled

full_data_upsampled = upsample(full_data, 'EndDate')
full_data_upsampled

,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,duration_contract,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
customerID,,,,,,,,,,,,,,,,,,
6900-PXRMS,1,0,0,2,0.890526,0.361022,1,1,1,1,1,0,1,1,0,1,1,1
9430-NKQLY,0,2,1,1,0.211368,1.000000,1,0,1,1,2,0,0,0,0,0,0,1
3746-EUBYR,1,0,1,0,0.628211,0.014149,1,0,1,0,1,0,0,0,0,0,0,1
0094-OIFMO,1,0,1,2,0.800000,0.152898,0,1,0,0,1,0,1,0,0,1,1,0
4391-RESHN,1,0,1,3,0.876211,0.319489,1,0,0,0,1,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8319-QBEHW,0,1,1,0,0.336421,0.361479,1,0,0,1,0,0,1,0,0,1,0,2
6918-UMQCG,0,0,0,2,0.675368,0.069831,0,0,0,0,1,0,0,0,0,1,0,0
7813-ZGGAW,0,0,1,0,0.813474,0.431310,1,1,0,0,1,0,0,1,1,1,0,1


In [70]:
full_data_upsampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10781 entries, 6900-PXRMS to 2330-PQGDQ
Data columns (total 18 columns):
EndDate              10781 non-null int64
Type                 10781 non-null int64
PaperlessBilling     10781 non-null int64
PaymentMethod        10781 non-null int64
MonthlyCharges       10781 non-null float64
duration_contract    10781 non-null float64
gender               10781 non-null int64
SeniorCitizen        10781 non-null int64
Partner              10781 non-null int64
Dependents           10781 non-null int64
InternetService      10781 non-null int64
OnlineSecurity       10781 non-null int64
OnlineBackup         10781 non-null int64
DeviceProtection     10781 non-null int64
TechSupport          10781 non-null int64
StreamingTV          10781 non-null int64
StreamingMovies      10781 non-null int64
MultipleLines        10781 non-null int64
dtypes: float64(2), int64(16)
memory usage: 1.6+ MB


In [71]:
df_train, df_valid_and_test = train_test_split(full_data_upsampled, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345)

features_train = df_train.drop(['EndDate'], axis=1)
target_train = df_train['EndDate']

features_valid = df_valid.drop(['EndDate'], axis=1)
target_valid =  df_valid['EndDate']

features_test = df_test.drop(['EndDate'], axis=1)
target_test =  df_test['EndDate']

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [74]:
%%time

model = CatBoostClassifier(iterations=20000,custom_loss=['AUC', 'Accuracy'])
model.fit(features_train, target_train, cat_features=cat_features, verbose=25)
probabilities_valid = model.predict(features_valid)

AUC_ROC = roc_auc_score(target_valid, probabilities_valid)
print('AUC_ROC =',AUC_ROC)

Learning rate set to 0.001969
0:	learn: 0.6922752	total: 11ms	remaining: 3m 39s
25:	learn: 0.6709761	total: 949ms	remaining: 12m 9s
50:	learn: 0.6510214	total: 2.43s	remaining: 15m 50s
75:	learn: 0.6334078	total: 3.81s	remaining: 16m 39s
100:	learn: 0.6169021	total: 5.3s	remaining: 17m 23s
125:	learn: 0.6025355	total: 6.38s	remaining: 16m 47s
150:	learn: 0.5897367	total: 7.46s	remaining: 16m 20s
175:	learn: 0.5780105	total: 8.55s	remaining: 16m 3s
200:	learn: 0.5673274	total: 9.46s	remaining: 15m 31s
225:	learn: 0.5581219	total: 10.4s	remaining: 15m 12s
250:	learn: 0.5499338	total: 11.4s	remaining: 14m 59s
275:	learn: 0.5421253	total: 12.8s	remaining: 15m 15s
300:	learn: 0.5353225	total: 14.2s	remaining: 15m 28s
325:	learn: 0.5292741	total: 15.5s	remaining: 15m 33s
350:	learn: 0.5235880	total: 16.5s	remaining: 15m 21s
375:	learn: 0.5182520	total: 17.4s	remaining: 15m 10s
400:	learn: 0.5135823	total: 18.4s	remaining: 15m 1s
425:	learn: 0.5092643	total: 19.4s	remaining: 14m 52s
450:	lear

3725:	learn: 0.4033165	total: 2m 57s	remaining: 12m 56s
3750:	learn: 0.4029624	total: 2m 59s	remaining: 12m 56s
3775:	learn: 0.4027229	total: 3m	remaining: 12m 54s
3800:	learn: 0.4022987	total: 3m 1s	remaining: 12m 52s
3825:	learn: 0.4020163	total: 3m 2s	remaining: 12m 51s
3850:	learn: 0.4017349	total: 3m 3s	remaining: 12m 50s
3875:	learn: 0.4013707	total: 3m 4s	remaining: 12m 48s
3900:	learn: 0.4009700	total: 3m 5s	remaining: 12m 46s
3925:	learn: 0.4006916	total: 3m 6s	remaining: 12m 44s
3950:	learn: 0.4004147	total: 3m 7s	remaining: 12m 43s
3975:	learn: 0.4001368	total: 3m 8s	remaining: 12m 41s
4000:	learn: 0.3997742	total: 3m 10s	remaining: 12m 39s
4025:	learn: 0.3994760	total: 3m 11s	remaining: 12m 37s
4050:	learn: 0.3991624	total: 3m 12s	remaining: 12m 38s
4075:	learn: 0.3988559	total: 3m 14s	remaining: 12m 38s
4100:	learn: 0.3984896	total: 3m 15s	remaining: 12m 37s
4125:	learn: 0.3982500	total: 3m 16s	remaining: 12m 35s
4150:	learn: 0.3978551	total: 3m 17s	remaining: 12m 33s
4175

7425:	learn: 0.3428787	total: 6m 4s	remaining: 10m 17s
7450:	learn: 0.3425204	total: 6m 5s	remaining: 10m 16s
7475:	learn: 0.3422274	total: 6m 7s	remaining: 10m 15s
7500:	learn: 0.3418599	total: 6m 8s	remaining: 10m 13s
7525:	learn: 0.3415214	total: 6m 9s	remaining: 10m 12s
7550:	learn: 0.3411807	total: 6m 11s	remaining: 10m 11s
7575:	learn: 0.3408466	total: 6m 12s	remaining: 10m 10s
7600:	learn: 0.3405240	total: 6m 13s	remaining: 10m 9s
7625:	learn: 0.3401393	total: 6m 14s	remaining: 10m 8s
7650:	learn: 0.3397936	total: 6m 15s	remaining: 10m 6s
7675:	learn: 0.3393664	total: 6m 17s	remaining: 10m 5s
7700:	learn: 0.3389941	total: 6m 18s	remaining: 10m 4s
7725:	learn: 0.3386860	total: 6m 19s	remaining: 10m 3s
7750:	learn: 0.3384134	total: 6m 20s	remaining: 10m 2s
7775:	learn: 0.3380522	total: 6m 22s	remaining: 10m
7800:	learn: 0.3376009	total: 6m 23s	remaining: 9m 59s
7825:	learn: 0.3372343	total: 6m 24s	remaining: 9m 58s
7850:	learn: 0.3368735	total: 6m 25s	remaining: 9m 57s
7875:	learn

11175:	learn: 0.2962442	total: 9m 14s	remaining: 7m 17s
11200:	learn: 0.2960316	total: 9m 15s	remaining: 7m 16s
11225:	learn: 0.2957678	total: 9m 17s	remaining: 7m 15s
11250:	learn: 0.2955732	total: 9m 18s	remaining: 7m 14s
11275:	learn: 0.2952954	total: 9m 19s	remaining: 7m 12s
11300:	learn: 0.2950236	total: 9m 20s	remaining: 7m 11s
11325:	learn: 0.2947516	total: 9m 21s	remaining: 7m 10s
11350:	learn: 0.2945640	total: 9m 23s	remaining: 7m 9s
11375:	learn: 0.2943277	total: 9m 24s	remaining: 7m 7s
11400:	learn: 0.2939707	total: 9m 25s	remaining: 7m 6s
11425:	learn: 0.2937228	total: 9m 26s	remaining: 7m 5s
11450:	learn: 0.2935031	total: 9m 28s	remaining: 7m 4s
11475:	learn: 0.2932010	total: 9m 29s	remaining: 7m 2s
11500:	learn: 0.2929420	total: 9m 30s	remaining: 7m 1s
11525:	learn: 0.2927095	total: 9m 31s	remaining: 7m
11550:	learn: 0.2924171	total: 9m 33s	remaining: 6m 59s
11575:	learn: 0.2921916	total: 9m 34s	remaining: 6m 57s
11600:	learn: 0.2919114	total: 9m 35s	remaining: 6m 56s
116

14825:	learn: 0.2623853	total: 12m 24s	remaining: 4m 19s
14850:	learn: 0.2621583	total: 12m 25s	remaining: 4m 18s
14875:	learn: 0.2619074	total: 12m 26s	remaining: 4m 17s
14900:	learn: 0.2617128	total: 12m 28s	remaining: 4m 16s
14925:	learn: 0.2615292	total: 12m 29s	remaining: 4m 14s
14950:	learn: 0.2613879	total: 12m 30s	remaining: 4m 13s
14975:	learn: 0.2612137	total: 12m 31s	remaining: 4m 12s
15000:	learn: 0.2610271	total: 12m 33s	remaining: 4m 10s
15025:	learn: 0.2608034	total: 12m 34s	remaining: 4m 9s
15050:	learn: 0.2605846	total: 12m 35s	remaining: 4m 8s
15075:	learn: 0.2603974	total: 12m 36s	remaining: 4m 7s
15100:	learn: 0.2601724	total: 12m 38s	remaining: 4m 5s
15125:	learn: 0.2599823	total: 12m 39s	remaining: 4m 4s
15150:	learn: 0.2597818	total: 12m 40s	remaining: 4m 3s
15175:	learn: 0.2596243	total: 12m 41s	remaining: 4m 2s
15200:	learn: 0.2594070	total: 12m 43s	remaining: 4m
15225:	learn: 0.2592211	total: 12m 44s	remaining: 3m 59s
15250:	learn: 0.2590171	total: 12m 45s	rem

18450:	learn: 0.2367953	total: 15m 32s	remaining: 1m 18s
18475:	learn: 0.2366338	total: 15m 34s	remaining: 1m 17s
18500:	learn: 0.2364466	total: 15m 35s	remaining: 1m 15s
18525:	learn: 0.2363101	total: 15m 36s	remaining: 1m 14s
18550:	learn: 0.2361325	total: 15m 37s	remaining: 1m 13s
18575:	learn: 0.2360154	total: 15m 39s	remaining: 1m 11s
18600:	learn: 0.2358582	total: 15m 40s	remaining: 1m 10s
18625:	learn: 0.2356671	total: 15m 41s	remaining: 1m 9s
18650:	learn: 0.2355109	total: 15m 43s	remaining: 1m 8s
18675:	learn: 0.2353582	total: 15m 44s	remaining: 1m 6s
18700:	learn: 0.2352118	total: 15m 45s	remaining: 1m 5s
18725:	learn: 0.2350441	total: 15m 47s	remaining: 1m 4s
18750:	learn: 0.2349059	total: 15m 48s	remaining: 1m 3s
18775:	learn: 0.2347883	total: 15m 49s	remaining: 1m 1s
18800:	learn: 0.2346548	total: 15m 50s	remaining: 1m
18825:	learn: 0.2345044	total: 15m 52s	remaining: 59.4s
18850:	learn: 0.2343613	total: 15m 53s	remaining: 58.1s
18875:	learn: 0.2341948	total: 15m 54s	remai

In [75]:
probabilities_test = model.predict(features_test)

AUC_ROC = roc_auc_score(target_test, probabilities_test)
print('AUC_ROC =',AUC_ROC)

AUC_ROC = 0.8718998762405017


##### Чтобы выполнить финальный спринт успешно, нужно набрать пять очков, или story points (sp)

Критерии оценки:

- AUC-ROC < 0.75 — 0 sp
- 0.75 ≤ AUC-ROC < 0.81 — 4 sp
- 0.81 ≤ AUC-ROC < 0.85 — 4.5 sp
##### - 0.85 ≤ AUC-ROC < 0.87 — 5 sp
- 0.87 ≤ AUC-ROC < 0.88 — 5.5 sp
- AUC-ROC ≥ 0.88 — 6 sp

Вывод:
В итоге удалось получить модель с отличными метриками. А всё благодаря чему? Отличной библиотеки CatBoostClassifier от Яндекс и обучению на Data Sciense от Яндекс Практикум. Ура!

# Отчёт:

##### Плат отчёта, на какие вопросы нужно ответить:

Какие пункты плана были выполнены, а какие — нет (поясните почему)?

Какие трудности возникли и как вы их преодолели?

Какие ключевые шаги в решении задачи выделили?

Какая ваша итоговая модель и какое у неё качество?

### План был такой:

1. Проанализировать датасеты, которые нам предоставили
2. Понять, как склеивать таблицы и что делать с пропусками, которые образуются. Понять - как лучше заполнить пропуски.
3. Понять, как предобработать данные для улучшения целевой метрики у модели (маштабирование, определение категориальных признаков, сделать балансировку классов)
4. Аналитически выбрать наиболее подходящую/ие модель/модели для данной задачи.
5. Обучить модель/модели, перебирать гиперапараметры, чтобы добиться лучшего значения метрики.

##### Анализ выполнения пунктов плана:

1. Мы внимательно проанализировали датасеты, которые нам были даны. С помощью подсказок Таймлида и более детального описания код-ревьювера, узнали, какие дополнительные полезные признаки можно выделить, и какие нужно удалить для предотвращения слива данных.

2. Мы легко скреили датасеты через присваивание новых индексов, равных столбцу 'customerID'. Исходя из описания датасетов, мы постарались подумать как именно заполнить образовавщиеся пропуски, чтобы это было наиболее логично.

3. Мы провели работу на сбалансировку классов в  датасете, путём увеличения количества строчек в датасете от класса, который занимал меньше строчек. Так же провели маштабирование признаков и логически определили какие признаки являются категориальными.

4. Нам не потребовалось перебирать модели в поисках наболее лучшей, так как первая библеотека, которую было решено использовать дала отличные результаты, для набора необходимого количества баллов. Учитывая, что время, которое необходимо потратить для получения модели - это то, что тоже требуется оптимизировать постоянно. Было решено не пробовать модели от других библиотек.

5. Я почитал документацию, и начал тестировать один гиперпараметр за другим, целевое значение метрики было достигнуто быстро, и не потребовалось перебирать много гиперпараметров. В реальном проекте, учитывая, как быстро я получил результаты (на работу потраченно менее 10 часов), я бы потратил намного больше времени для получения более высокой метрики, в том числе и ради интереса. Но всегда интереснее делать проекты для реальных задачь. Поэтому было решено остановиться лишь на нескольких гиперпараметрах, так как целевая метрика была достигнута.

##### Вывод:
Все пункты плана были выполнены. План был достаточно стандартный.

Сильных трудностей не возникало. В яндекс очень хорошее обучение, на порядок выше любого другого, где я учился (университет, например), а благодаря хорошему обучению, и умению гуглить - работа была выполнена очень быстро и как мне кажется достаточно качественно.

Самую большую часть времени я потратил на обработку датасетов, и вникание в то, как заполнять пропуски и какие признаки можно удалять.

Моя итоговая модель - единственная модель, какую я создал, у неё метрика качества:
1) AUC-ROC на валидационной выборке 0.87 (с округлением до сотых)
2) AUC-ROC на тестовой выборке 0.87